# Training and Converting an LSTM Model from PyTorch to Tensorflow Lite for Micro


## Training in PyTorch

Let's train a hand-written character recognition model with the MNIST dataset which uses LSTM models.

Start off with importing the dataset:

In [1]:
import torch
import torchvision
from torchvision import transforms

"""
Each data point is of form [PIL.Image, class] where the class is a number 0-9 of the
respective digit. The image is a single channel image with values from 0 to 1.
"""
def mnist_data(train: bool) -> torchvision.datasets.MNIST:
    return torchvision.datasets.MNIST(
        './mnist/',
        download=True,
        train=train, 
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
)

# iterator returning Tensors of type [batch (10), channels (1), height (28), width (28)]
data_loaders = {
    "train": torch.utils.data.DataLoader(
        mnist_data(True),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),
    "test": torch.utils.data.DataLoader(
        mnist_data(False),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),}

### Define Model

In [24]:
class MNIST_LSTM(torch.nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int = 1):
        super(MNIST_LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        
        self.lstm = torch.nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True
        )
        self.dropout = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(hidden_size, 10)
        
    def forward(self, x):
        
        # Remove batch parameter
        x = x.reshape(-1, self.input_size, self.input_size)

        # Set initial hidden and cell states 
        hidden_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        cell_states_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        lstm_out, hidden = self.lstm(x, (hidden_initial, cell_states_initial))
        
        dropout_out = self.dropout(lstm_out)
        
        fc_out = self.fc(dropout_out[:, -1, :])
        return fc_out
       

### Training Loop

In [26]:
input_size=28

# instantiate model
model = MNIST_LSTM(input_size, hidden_size=128)

loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [27]:
def train(num_epochs: int, model: MNIST_LSTM, loaders):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for i, (images, labels) in enumerate(loaders['train']):
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    print("Ended epoch: ")

In [28]:
train(num_epochs=3, model=model, loaders=data_loaders)

Epoch 0/3
Epoch 1/3
Epoch 2/3
Ended epoch: 


In [30]:
# Test the model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in data_loaders['test']:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total = total + labels.size(0)
        
        correct = correct + (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.07 %


### Model Usage

In [31]:
images, labels = next(iter(data_loaders["test"]))

results = model(images).tolist()
processed_results = list(map(lambda res: res.index(max(res)), results))

display(torchvision.transforms.ToPILImage()(images[0]))
print(processed_results[0])

7


## Convert to ONNX

In [116]:
sample_input, sample_labels = next(iter(data_loaders["test"]))

onnx_filename = "handwriting.onnx"

torch.onnx.export(
    model,
    sample_input,
    onnx_filename,
    input_names= ["image"],
    output_names=["number"]
)

## Test ONNX

In [35]:
import onnx

# Load the ONNX model
onnx_model = onnx.load(onnx_filename)

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [36]:
import numpy
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_filename)

outputs = ort_session.run(
    None,
    {"image": sample_input.numpy()},
)
onnx_processed_results = [numpy.argmax(res) for res in outputs[0]]

print(onnx_processed_results[0])
display(torchvision.transforms.ToPILImage()(sample_input[0]))

3



## ONNX To TFlite

### First to Keras

In [145]:
import onnx2tf

keras_model = onnx2tf.convert(
    input_onnx_file_path="handwriting.onnx",
    output_folder_path="mnist.tf",
    copy_onnx_input_output_names_to_tflite=True,
    non_verbose=True,
    param_replacement_file="param_replacement.json"
)

#### Check Keras Accuracy

In [ ]:
keras_correct = 0
keras_total = 0

for test_images, test_labels in data_loaders['test']:
    #transpose the input_batch into BHWC order for tensorflow
    tf_input_data = np.transpose(test_images.numpy(), [0, 2, 3, 1])

    keras_output_data = keras_model(tf_input_data)
    processed_keras_output = [numpy.argmax(res) for res in keras_output_data]
    
    keras_results = zip(processed_keras_output, test_labels)
    keras_batch_correct = sum([1 if output == label.item() else 0 for output, label in keras_results])
        
    keras_correct = keras_correct + keras_batch_correct
    keras_total = keras_total + test_labels.size(0)


print(f"Test Accuracy of the keras model on the {keras_total} test images: {100 * keras_correct / keras_total} % ({keras_correct}/{keras_total})")

### Now exoirt as TensorFlow Lite for Micro
Now let's quantise our model to int8 and export it to a TensorFlow Lite flatbuffer.


#### Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to Converting a keras model into an xcore optimised tflite model for more details on this.


In [149]:
def representative_dataset():
    for test_images, _ in data_loaders['test']:
        tf_batch = np.transpose(test_images.numpy(), [0, 2, 3, 1])
        yield [tf_batch]

In [150]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'handwriting.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


In [167]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

tfl_correct = 0
tfl_total = 0

for test_images, test_labels in data_loaders['test']:
    #transpose the input_batch into BHWC order for tensorflow
    tfl_input_data = np.transpose(test_images.numpy(), [0, 2, 3, 1])

    tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tfl_input_data)
    tfl_interpreter.invoke()

    tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

    tfl_processed_results = [numpy.argmax(res) for res in tfl_output_data]
    
    tfl_results = zip(tfl_processed_results, test_labels)
    tfl_batch_correct = sum([1 if output == label.item() else 0 for output, label in tfl_results])
        
    tfl_correct = tfl_correct + tfl_batch_correct
    tfl_total = tfl_total + test_labels.size(0)
    
    if tfl_total % 500 == 0:
        print(f"running accuracy: {tfl_correct}/{tfl_total} ({100 * tfl_correct / tfl_total}%)")

print(f"Test Accuracy of the TensorFlow Lite for Micro model on the {tfl_total} test images: {100 * tfl_correct / tfl_total} % ({tfl_correct}/{tfl_total})")

running accuracy: 488/500 (97.6%)
running accuracy: 982/1000 (98.2%)
running accuracy: 1473/1500 (98.2%)
running accuracy: 1963/2000 (98.15%)
running accuracy: 2453/2500 (98.12%)
running accuracy: 2946/3000 (98.2%)
running accuracy: 3430/3500 (98.0%)
running accuracy: 3922/4000 (98.05%)
running accuracy: 4415/4500 (98.11111111111111%)
running accuracy: 4903/5000 (98.06%)
running accuracy: 5396/5500 (98.10909090909091%)
running accuracy: 5890/6000 (98.16666666666667%)
running accuracy: 6383/6500 (98.2%)
running accuracy: 6872/7000 (98.17142857142858%)
running accuracy: 7358/7500 (98.10666666666667%)
running accuracy: 7842/8000 (98.025%)
running accuracy: 8333/8500 (98.03529411764706%)
running accuracy: 8822/9000 (98.02222222222223%)
running accuracy: 9310/9500 (98.0%)
running accuracy: 9798/10000 (97.98%)
Test Accuracy of the TensorFlow Lite for Micro model on the 10000 test images: 97.98 % (9798/10000)


## Training in TensorFlow


In [ ]:
import keras
from keras.datasets import mnist

#Importing the data
(X_train, y_train),(X_test, y_test) = mnist.load_data() 

#Normalizing the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [ ]:
X_train.shape[1:]

In [ ]:

#Initializing model
model = keras.models.Sequential()

#Adding the model layers
model.add(keras.layers.LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10, activation='softmax'))

#Compiling the model
model.compile( loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam(lr=0.001, decay=1e-6), metrics=['accuracy'] )

#Fitting data to the model
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

In [ ]:
# Evaluating the test loss and accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
